# Sankey diagram of connections in trading between continents

This can be used in the future to show trading between continents and emissions consequences.

In [1]:
#Use python 3.11
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns

In [2]:
# Load data
df = pd.read_csv(r'.\data\continent_trade_matrix_CO2eq.csv')

In [3]:
print(df.year.max(), df.year.min())

2022 1986


In [4]:
#Drop antarctica
df = df[df['target']!= 'antarctica']
df = df[df['source']!= 'antarctica']

In [5]:
unique_continents = sorted(set(df['source'].unique()) | set(df['target'].unique()))

In [6]:
# Generate color palette
palette = sns.color_palette("colorblind", len(unique_continents))
# Convert Seaborn colors to hex colors
hex_colors = palette.as_hex()

palette

[(0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.47058823529411764, 0.7372549019607844),
 (0.792156862745098, 0.5686274509803921, 0.3803921568627451)]

In [20]:
# Assuming your DataFrame is named 'df'
year = 2022
df_year = df[df['year'] == year]

# Extract unique continents for labels
unique_continents = sorted(set(df_year['source'].unique()) | set(df_year['target'].unique()))

# Create mappings to node indices (0, 1, 2, ...)
continent_to_index = {continent: i for i, continent in enumerate(unique_continents)}

# Prepare data for Sankey diagram
nodes = dict(label=unique_continents, color= hex_colors)  # Customize colors
links = dict(
    source=df_year['source'].map(continent_to_index),
    target=df_year['target'].map(continent_to_index),
    value=df_year['value'],
)

# Adjust link colors based on source node color
opacity = 0.4
link_colors = [nodes['color'][src].replace(")", f", {opacity})") for src in links['target']]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node= nodes,  # Customize colors,
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        color=link_colors  # Use adjusted colors
    )
)])

fig.update_layout(
    title_text=f"{year} kg CO2eq Produced by Food Trade by Continent",
    font_size=16,
    )

fig.show()


# One plot every 10 years from 2022 back

In [21]:
# Assuming your DataFrame is named 'df'
# Create a list of years from 2022 backwards in steps of 10 years
years = list(range(2022, df['year'].min() - 1, -10))

# Filter out years that are not in the DataFrame
years = [year for year in years if year in df['year'].unique()]

# Loop through each year and create Sankey diagrams
for year in years:
    df_year = df[df['year'] == year]

    # Extract unique continents for labels
    unique_continents = sorted(set(df_year['source'].unique()) | set(df_year['target'].unique()))

    # Create mappings to node indices (0, 1, 2, ...)
    continent_to_index = {continent: i for i, continent in enumerate(unique_continents)}

    # Prepare data for Sankey diagram
    nodes = dict(label=unique_continents, color=hex_colors)  # Customize colors
    links = dict(
        source=df_year['source'].map(continent_to_index),
        target=df_year['target'].map(continent_to_index),
        value=df_year['value'],
    )

    # Adjust link colors based on source node color
    opacity = 0.4
    link_colors = [nodes['color'][src].replace(")", f", {opacity})") for src in links['target']]

    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=nodes,  # Customize colors,
        link=dict(
            source=links['source'],
            target=links['target'],
            value=links['value'],
            color=link_colors  # Use adjusted colors
        )
    )])

    fig.update_layout(
        title_text=f"{year} kg CO2eq Produced by Food Trade by Continent",
        font_size=16,
    )

    # Save each figure to an HTML file
    fig.write_html(f"sankey_{year}.html")